# Material Didático: Criando um Sistema de Recomendação Explicável (XRS)

**Objetivo:** Sair da "Caixa-Preta".

Neste tutorial, vamos construir um Sistema de Recomendação (SR) que não apenas sugere itens, mas também explica *por que* fez essa sugestão. [cite_start]A maioria dos SRs sofre com a percepção de serem "caixas-pretas", o que gera desconfiança [cite: 104-106]. Nosso objetivo é criar um sistema transparente.

**A Abordagem: Explicação por "Vizinhos Similares"**

Usaremos um dos métodos de recomendação mais clássicos, a **Filtragem Colaborativa Baseada em Usuário (User-Based KNN)**. [cite_start]A própria lógica desse algoritmo nos dá a explicação, como planejado na nossa proposta de projeto :

> "Recomendamos este filme para você porque **usuários com gostos parecidos com o seu** (seus 'vizinhos') também gostaram dele."

**Ferramentas que usaremos:**
* [cite_start]**Dataset:** MovieLens 100k (um dataset público clássico)[cite: 119].
* [cite_start]**Recomendação:** Biblioteca `CaseRecommender` (para gerar a predição da nota)[cite: 126].
* **Explicação:** Bibliotecas `Pandas` e `Scikit-learn` (para "abrir" o modelo e encontrar os vizinhos que geraram a recomendação).

In [8]:
# CÉLULA 1: SETUP E INSTALAÇÃO

# 1. Instalar a biblioteca CaseRecommender direto do GitHub
print("Instalando o CaseRecommender...")
!pip install "numpy<2.0" #caserec está desatualizada
!pip install CaseRecommender

# 2. Baixar o dataset MovieLens 100k
print("Baixando o dataset MovieLens 100k...")
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip -q
!unzip ml-100k.zip -d ml-100k
!mv ml-100k/ml-100k/* .

print("\nSetup concluído!")
print("Bibliotecas instaladas e dataset baixado.")

Instalando o CaseRecommender...
Baixando o dataset MovieLens 100k...
Archive:  ml-100k.zip
  inflating: ml-100k/ml-100k/allbut.pl  
  inflating: ml-100k/ml-100k/mku.sh  
  inflating: ml-100k/ml-100k/README  
  inflating: ml-100k/ml-100k/u.data  
  inflating: ml-100k/ml-100k/u.genre  
  inflating: ml-100k/ml-100k/u.info  
  inflating: ml-100k/ml-100k/u.item  
  inflating: ml-100k/ml-100k/u.occupation  
  inflating: ml-100k/ml-100k/u.user  
  inflating: ml-100k/ml-100k/u1.base  
  inflating: ml-100k/ml-100k/u1.test  
  inflating: ml-100k/ml-100k/u2.base  
  inflating: ml-100k/ml-100k/u2.test  
  inflating: ml-100k/ml-100k/u3.base  
  inflating: ml-100k/ml-100k/u3.test  
  inflating: ml-100k/ml-100k/u4.base  
  inflating: ml-100k/ml-100k/u4.test  
  inflating: ml-100k/ml-100k/u5.base  
  inflating: ml-100k/ml-100k/u5.test  
  inflating: ml-100k/ml-100k/ua.base  
  inflating: ml-100k/ml-100k/ua.test  
  inflating: ml-100k/ml-100k/ub.base  
  inflating: ml-100k/ml-100k/ub.test  

Setup conc

## Parte 1: Gerando a Recomendação (A "Caixa-Preta")

Nesta primeira parte, vamos atuar como um sistema tradicional. Nosso objetivo é simplesmente treinar um modelo que receba um `user_id` e nos diga qual filme ele deveria assistir.

Usaremos o `CaseRecommender` para treinar um modelo `UserBasedKNN` (K-Nearest Neighbors, ou K-Vizinhos Mais Próximos). Este modelo prevê a nota que um usuário daria a um filme com base nas notas que usuários "vizinhos" (similares) deram.

In [27]:
# CÉLULA 2: IMPORTS E PREPARAÇÃO DOS DADOS

import pandas as pd
from caserec.recommenders.item_recommendation.userknn import UserKNN

# --- Carregar os dados ---

# 1. Carregar 'u.data': o arquivo de avaliações
# Formato: user_id | item_id | rating | timestamp
print("Carregando avaliações (u.data)...")
ratings_df = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# 2. Carregar 'u.item': o arquivo de metadados dos filmes
# Precisamos dele para saber os nomes dos filmes
# O encoding 'latin-1' é necessário por causa de caracteres especiais nos títulos
print("Carregando títulos dos filmes (u.item)...")
movies_df = pd.read_csv('u.item', sep='|', encoding='latin-1', header=None,
                        names=['item_id', 'title'] + list(range(2, 24)))[['item_id', 'title']]

# Usar o item_id como índice facilita a busca de nomes
movies_df.set_index('item_id', inplace=True)


# --- Definir arquivos de treino e teste ---

# O MovieLens 100k já vem com splits de treino/teste pré-definidos (u1.base, u1.test, etc.)
# Isso é uma boa prática para a reprodutibilidade. Vamos usar o split 1.
train_file = 'u1.base'
test_file = 'u1.test' # Usando o u.test completo para o exemplo

print("\nDados carregados e prontos.")
print(f"Temos {ratings_df['user_id'].nunique()} usuários e {movies_df.shape[0]} filmes no total.")

Carregando avaliações (u.data)...
Carregando títulos dos filmes (u.item)...

Dados carregados e prontos.
Temos 943 usuários e 1682 filmes no total.


### 1.1 Treinando o Modelo

Agora, vamos alimentar o `CaseRecommender` com nossos arquivos de treino e teste. O algoritmo `UserBasedKNN` vai calcular a similaridade entre os usuários (usando o `train_file`) e, em seguida, gerar predições para todos os pares (usuário, item) no `test_file`.

Ele salvará o resultado em um novo arquivo: `predicoes.dat`.

In [15]:
# CÉLULA 3: TREINAR O MODELO DE RECOMENDAÇÃO (UserBasedKNN)

print("Treinando o modelo UserBasedKNN...")
# k_neighbors=30: Usar os 30 vizinhos mais próximos para a predição
# as_similar_first=True: Otimização específica para UserKNN
UserKNN(train_file, test_file, 'predicoes.dat', k_neighbors=30, as_similar_first=True).compute()

print("\nModelo treinado com sucesso!")
print("As predições de nota foram salvas em 'predicoes.dat'")

Treinando o modelo UserBasedKNN...
[Case Recommender: Item Recommendation > UserKNN Algorithm]

train data:: 943 users and 1650 items (80000 interactions) | sparsity:: 94.86%
test data:: 459 users and 1410 items (20000 interactions) | sparsity:: 96.91%

training_time:: 0.868679 sec
prediction_time:: 12.271117 sec


Eval:: PREC@1: 0.64488 PREC@3: 0.581699 PREC@5: 0.538126 PREC@10: 0.469935 RECALL@1: 0.027808 RECALL@3: 0.074558 RECALL@5: 0.106723 RECALL@10: 0.175355 MAP@1: 0.64488 MAP@3: 0.721859 MAP@5: 0.709786 MAP@10: 0.660293 NDCG@1: 0.64488 NDCG@3: 0.79523 NDCG@5: 0.781037 NDCG@10: 0.756958 

Modelo treinado com sucesso!
As predições de nota foram salvas em 'predicoes.dat'


### 1.2 Vendo as Recomendações

O arquivo `predicoes.dat` contém as notas previstas, mas isso não é uma lista de recomendação. Para criar uma lista útil, precisamos:

1.  Escolher um **usuário-alvo** (ex: `user_id = 1`).
2.  Carregar as predições para esse usuário.
3.  Adicionar os nomes dos filmes para que a lista seja legível.
4.  **Crucial:** Remover todos os filmes que o usuário **já assistiu**. Não se recomenda o que já foi visto!
5.  Ordenar a lista pela maior nota prevista.

In [17]:
# CÉLULA 4: VER AS RECOMENDAÇÕES PARA UM USUÁRIO ALVO

# Vamos focar no Usuário 1 (um usuário de exemplo)
user_id_alvo = 1

# Carregar as predições feitas pelo CaseRecommender
predicoes_df = pd.read_csv('predicoes.dat', sep='\t', names=['user_id', 'item_id', 'rating_pred'])

# 1. Filtrar apenas as predições para o nosso usuário alvo
recs_usuario_alvo = predicoes_df[predicoes_df['user_id'] == user_id_alvo]

# 2. Adicionar os nomes dos filmes
recs_usuario_alvo = recs_usuario_alvo.join(movies_df, on='item_id')

# 3. Remover filmes que o Usuário 1 JÁ VIU
# Primeiro, pegamos a lista de todos os filmes que o usuário 1 já viu (do 'u.data' original)
vistos_df = ratings_df[ratings_df['user_id'] == user_id_alvo]
itens_vistos = vistos_df['item_id'].tolist()

# Agora, filtramos a tabela de recomendações, mantendo apenas itens que NÃO ESTÃO na lista 'itens_vistos'
recs_usuario_alvo = recs_usuario_alvo[~recs_usuario_alvo['item_id'].isin(itens_vistos)]

# 4. Ordenar pelas melhores predições (notas mais altas)
recs_usuario_alvo = recs_usuario_alvo.sort_values('rating_pred', ascending=False)

print(f"\n--- 10 Melhores Recomendações (Filmes que o Usuário {user_id_alvo} NÃO VIU) ---")
print(recs_usuario_alvo[['title', 'rating_pred']].head(10))

# Salvar a recomendação TOP 1 para podermos explicar
top_1_rec = recs_usuario_alvo.iloc[0]
top_1_rec_id = int(top_1_rec['item_id'])
top_1_rec_title = top_1_rec['title']


--- 10 Melhores Recomendações (Filmes que o Usuário 1 NÃO VIU) ---
                               title  rating_pred
3  E.T. the Extra-Terrestrial (1982)    10.037096
9                 Stand by Me (1986)     9.299199


## Parte 2: "Abrindo a Caixa-Preta" (A Explicação)

Temos nossa recomendação principal: O `Usuário 1` deveria assistir ao filme **`E.T. the Extra-Terrestrial`**, pois prevemos que ele daria uma nota alta.

**Mas por quê?**

É aqui que o XRS entra. Vamos "recriar" a lógica do algoritmo para encontrar a resposta. O `UserBasedKNN` funciona assim:
1.  Encontra os **"vizinhos"** do Usuário 1 (outros usuários que avaliaram filmes de forma parecida no passado).
2.  Olha como esses vizinhos avaliaram o `E.T. the Extra-Terrestrial`.
3.  Calcula uma média ponderada dessas notas para prever a nota do Usuário 1.



Para criar nossa explicação, vamos usar `Pandas` e `Scikit-learn` para:
1.  Construir uma **Matriz Usuário-Item** com os dados de *treino*.
2.  Calcular a **Similaridade de Cosseno** entre todos os usuários.
3.  Encontrar os **Top 5 vizinhos** do nosso `user_id_alvo`.
4.  Verificar o que esses vizinhos acharam do filme recomendado.

In [18]:
# CÉLULA 5: IMPORTS PARA A EXPLICAÇÃO

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("Ferramentas de explicação (sklearn, numpy) importadas.")

Ferramentas de explicação (sklearn, numpy) importadas.


### 2.1 A Matriz Usuário-Item

Para calcular a similaridade, precisamos de um "perfil" de cada usuário. Fazemos isso criando uma matriz (tabela) gigante onde:
* Cada **linha** é um `user_id`.
* Cada **coluna** é um `item_id`.
* O **valor** é a `rating` (nota) que o usuário deu.

Usaremos apenas os dados de **treino** (`u1.base`), pois foi com eles que o modelo aprendeu.

In [19]:
# CÉLULA 6: CRIAR A MATRIZ DE SIMILARIDADE

print("Calculando a similaridade entre todos os usuários (pode levar alguns segundos)...")

# 1. Carregar o dataset de TREINO (u1.base)
train_df = pd.read_csv(train_file, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# 2. Criar a matriz Usuário-Item (pivotar a tabela)
user_item_matrix = train_df.pivot_table(index='user_id', columns='item_id', values='rating')

# 3. Preencher NaN (filmes não vistos) com 0
# A similaridade de cosseno precisa de valores numéricos, não de "buracos" (NaN)
user_item_matrix.fillna(0, inplace=True)

# 4. Calcular a Similaridade de Cosseno entre todos os usuários
# user_item_matrix.values são os dados brutos (apenas notas)
# O resultado é uma matriz (Usuário x Usuário) onde cada valor é a similaridade (de 0.0 a 1.0)
user_similarity_matrix = cosine_similarity(user_item_matrix)

# 5. Converter de volta para um DataFrame do Pandas para ficar fácil de ler
# (O índice/coluna será o user_id real, que começa em 1)
user_sim_df = pd.DataFrame(user_similarity_matrix,
                           index=user_item_matrix.index,
                           columns=user_item_matrix.index)

print("Matriz de similaridade (Usuário x Usuário) criada com sucesso.")
print("Exemplo da matriz (Usuário 1 vs. os 5 primeiros):")
print(user_sim_df.loc[1].head())

Calculando a similaridade entre todos os usuários (pode levar alguns segundos)...
Matriz de similaridade (Usuário x Usuário) criada com sucesso.
Exemplo da matriz (Usuário 1 vs. os 5 primeiros):
user_id
1    1.000000
2    0.097021
3    0.052469
4    0.021162
5    0.193545
Name: 1, dtype: float64


### 2.2 Encontrando os "Vizinhos"

Agora que temos a matriz `user_sim_df`, podemos "consultar" quem são os usuários mais parecidos com o nosso `user_id_alvo`.

In [20]:
# CÉLULA 7: ENCONTRAR OS "VIZINHOS" (USUÁRIOS SIMILARES)

# (Lembre-se: o user_id_alvo é 1)
print(f"Encontrando os vizinhos mais próximos do Usuário {user_id_alvo}...")

# 1. Pegar a "linha" (ou coluna) do Usuário 1 na matriz de similaridade
similaridades_usuario_alvo = user_sim_df[user_id_alvo]

# 2. Ordenar do mais similar (perto de 1.0) para o menos similar
# (E remover o próprio usuário 1 da lista, pois ele é 100% similar a si mesmo)
vizinhos = similaridades_usuario_alvo.drop(user_id_alvo).sort_values(ascending=False)

# 3. Pegar os 5 vizinhos mais similares
top_5_vizinhos = vizinhos.head(5)

print("\n--- Top 5 Usuários Mais Similares ao Usuário 1 ---")
print(top_5_vizinhos)

Encontrando os vizinhos mais próximos do Usuário 1...

--- Top 5 Usuários Mais Similares ao Usuário 1 ---
user_id
823    0.405729
514    0.394819
864    0.381792
592    0.371713
521    0.367758
Name: 1, dtype: float64


### 2.3 A Prova: Gerando a Explicação

Temos a nossa "prova"! Encontramos os 5 usuários mais parecidos com o Usuário 1.

O passo final é checar o que esses 5 vizinhos acharam do filme que recomendamos (`top_1_rec_title`).

Vamos procurar no nosso `train_df` (os dados de treino) e ver se eles assistiram a esse filme e qual nota deram.

In [28]:
# CÉLULA 8: VERIFICAR O QUE OS VIZINHOS ACHARAM

print(f"Verificando o que os Top 5 vizinhos acharam do filme: '{top_1_rec_title}' (ID: {top_1_rec_id})")

# Lista para guardar a "prova" da nossa explicação
vizinhos_que_gostaram = []

# Iterar sobre os 5 vizinhos que encontramos (o ID e o score de similaridade)
for vizinho_id, similaridade in top_5_vizinhos.items():

    # Checar se esse vizinho avaliou o filme recomendado
    # E se ele deu uma nota alta (ex: 4 ou 5)

    # 1. Pegar todas as avaliações daquele vizinho (do dataset de TREINO)
    ratings_vizinho = train_df[train_df['user_id'] == vizinho_id]

    # 2. Ver a nota específica para o filme recomendado (top_1_rec_id)
    nota_filme = ratings_vizinho[ratings_vizinho['item_id'] == top_1_rec_id]

    # 3. Se o vizinho realmente viu o filme E deu nota alta, guarde-o
    if not nota_filme.empty:
        nota = nota_filme.iloc[0]['rating']
        if nota >= 4:
            # Guardamos o ID, a nota, e o quão similar ele é
            vizinhos_que_gostaram.append((vizinho_id, nota, similaridade))

print("\nVerificação concluída.")

Verificando o que os Top 5 vizinhos acharam do filme: 'E.T. the Extra-Terrestrial (1982)' (ID: 423)

Verificação concluída.


## Conclusão: O Resultado Final (A Explicação)

Agora temos todos os componentes. Vamos formatar a saída de uma forma clara para nosso usuário final.

In [26]:
# CÉLULA 9: FORMATAR A EXPLICAÇÃO FINAL

print("========================================================")
print(f"Recomendação para o Usuário {user_id_alvo}:")
print(f"  Filme: {top_1_rec_title}")
print(f"  Nota prevista: {top_1_rec['rating_pred']:.2f} / 5.0")
print("========================================================\n")

# Se encontramos vizinhos que gostaram, usamos eles na explicação
if vizinhos_que_gostaram:
    print("✨ Por que recomendamos este filme? \n")
    print("Porque usuários com gostos muito parecidos com o seu gostaram dele:\n")

    for v_id, v_nota, v_sim in vizinhos_que_gostaram:
        sim_percent = v_sim * 100
        print(f"  - O Usuário {v_id} (que é {sim_percent:.1f}% similar a você) deu nota {v_nota}/5.")

# Se nenhum dos 5 principais vizinhos viu (ou gostou) do filme,
# damos uma explicação um pouco mais genérica.
else:
    print("✨ Por que recomendamos este filme? \n")
    print("Porque ele segue o padrão de filmes que usuários com gostos similares")
    print("aos seus costumam avaliar bem (mesmo que seus 5 'vizinhos' mais")
    print("próximos ainda não tenham avaliado este item específico).")

print("\n========================================================")

Recomendação para o Usuário 1:
  Filme: E.T. the Extra-Terrestrial (1982)
  Nota prevista: 10.04 / 5.0

✨ Por que recomendamos este filme? 

Porque usuários com gostos muito parecidos com o seu gostaram dele:

  - O Usuário 823 (que é 40.6% similar a você) deu nota 5/5.
  - O Usuário 514 (que é 39.5% similar a você) deu nota 5/5.
  - O Usuário 864 (que é 38.2% similar a você) deu nota 5/5.
  - O Usuário 592 (que é 37.2% similar a você) deu nota 5/5.



### O que aprendemos?

Este notebook demonstrou um ciclo completo de **Recomendação Explicável (XRS)**.

1.  **Geramos a Recomendação:** Usamos o `CaseRecommender` para encontrar um filme que o `Usuário 1` provavelmente gostaria.
2.  **Produzimos a Explicação:** Em vez de confiar em uma "caixa-preta", nós "abrimos" o modelo. Calculamos a matriz de similaridade, identificamos os vizinhos exatos do `Usuário 1`, e usamos as avaliações deles como a "prova" da nossa recomendação.

Este método de "vizinhos similares" é um exemplo de sistema que é **intrinsecamente explicável** (ou *explainable by design*), pois a própria lógica do algoritmo é a explicação. Este notebook agora pode ser usado como a base para o artigo final, o repositório do GitHub e a videoaula.